In [6]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sys

In [13]:
def proxy_finder_validate(item, candidates, df1, df2, predictors, orthogonal_vars):

    # validate proxies and st item
    assert item in df1.columns, f'AssertionError: item {item} not in df1.columns'

    assert predictors, f'AssertionError: missing predictors. If you would prefer to not specify predictors, do not pass in a variable.'
    
    for c in predictors:
        assert c in df1.columns, f'AssertionError: predictor {c} not in df1.columns'
        assert c in df2.columns, f'AssertionError: predictor {c} not in df2.columns' # we need same variable in second dataset  
        assert c in df1.select_dtypes(include=['number']).columns, f'predictor {c} is not a numeric column in df1'   
        assert c in df2.select_dtypes(include=['number']).columns, f'predictor {c} is not a numeric column in df2'    
    
    for c in candidates:
        assert c in df2.columns, f'AssertionError: candidate {c} not in df2.columns'
        
    if (orthogonal_vars != None):
        for c in orthogonal_vars:
            assert c in df2.columns, f'AssertionError: orthogonal variable {c} not in df2.columns'
                

In [14]:
# return a new df that is a copy of df, with: rescale all columns to be
#  between 0 and 1, inclusive. Drop any non-numeric columns. Drop any
# rows that are missing at least one predictor.
def data_rescale(df, predictors, target, drop=True):
    df = df.copy() # preserve immutability

    # Select only the numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns

    if drop:
      # drop any rows that are missing at least one predictor
      df = df.dropna(subset=predictors)
     
    # print('the dataframe we\'re rescaling is size: ') # debug
    # Initialize the scaler
    scaler = MinMaxScaler()

    # Fit the scaler to the data and transform it
    scaled_values = scaler.fit_transform(df[numeric_cols])

    # Create a new DataFrame with the scaled values, maintaining the original column names
    scaled_df = pd.DataFrame(scaled_values, columns=numeric_cols, index=df.index)

    return scaled_df

In [15]:
### GET PREDICTIONS

# Neural network definition
def build_nn_model(input_dim, learning_rate=0.001, l2_lambda=0.001):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,), kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        Dropout(0.5),  
        Dense(32, activation='relu', kernel_regularizer=l2(l2_lambda)),
        BatchNormalization(),
        Dropout(0.5),  
        Dense(1, kernel_regularizer=l2(l2_lambda))
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# return a trained neural network to predict df[item] using df[predictors_df1]
# report error and crash if predictors don't predict item
def train_nn_model(X_train, y_train, input_dim, epochs=100, learning_rate=0.001, l2_lambda=0.001):
    model = build_nn_model(input_dim, learning_rate, l2_lambda)
    model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, verbose=0)
    return model

# get predictions from the neural network. Takes in
def get_predictions(df_train, df_test, predictors, target, epochs=100, learning_rate=0.001, l2_lambda=0.001):
    
    # split data for training and testing. 
    X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(df_train[predictors].to_numpy(), df_train[target].to_numpy(), test_size=0.2, random_state=42)
    X_test = df_test[predictors].to_numpy()

    # train network and get predictions
    model = train_nn_model(X_train_train, y_train_train, len(predictors), epochs, learning_rate, l2_lambda)
    predictions = model.predict(X_test)

    # exit if correlation between predictions and item is bad
    mse = mean_squared_error(model.predict(X_train_test), y_train_test)
    print(f"Debug statement: Neural Net test MSE = {mse}") ####DEBUG
    if (mse > 0.03):
        print('Input Error: Predictors cannot predict {target} in df1', file=sys.stderr)
        print('Aborting program')
        sys.exit(-1)

   # print(f"Predictions before flattening: {predictions[:10]}") #DEBUG
   # print('predictions after flattening: ', predictions.flatten()[:10])#DEBUG

    return predictions.flatten()

# get predictions from the Torch neural network
def get_predictionsTorch(df_train, df_test, predictors, target, epochs=100, learning_rate=0.001, l2_lambda=0.001):
    import torch
    import torch.nn as nn
    import torch.optim as optim
    import torch.nn.functional as F

    # split data for training and testing.
    training_features, validation_features, training_target, validation_target = train_test_split(df_train[predictors].to_numpy(), df_train[target].to_numpy(), test_size=0.2, random_state=42)

    training_features = torch.FloatTensor(training_features)
    training_target = torch.FloatTensor(training_target)
    validation_features = torch.FloatTensor(validation_features)
    validation_target = torch.FloatTensor(validation_target)

    model = nn.Sequential(
        nn.Linear(len(predictors), 64),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(32, 1)
    )

    # Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

    # MSE loss
    loss_func = nn.MSELoss()

    # train the model
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()

        # Forward pass
        prediction = model(training_features)
        loss = loss_func(prediction, training_target.unsqueeze(1))

        # Backward pass
        loss.backward()
        optimizer.step()

    # get predictions
    model.eval()
    test_data = torch.FloatTensor(df_test[predictors].to_numpy())

    with torch.no_grad():
        predictions = model(test_data)
        predictions = predictions.numpy().flatten()

        val_predictions = model(validation_features)
        val_predictions = val_predictions.numpy().flatten()


    # exit if correlation between predictions and item is bad
    mse = mean_squared_error(val_predictions, validation_target)
    print(f"Debug statement: Neural Net test MSE = {mse}") ####DEBUG
    if (mse > 0.03):
        print('Input Error: Predictors cannot predict {target} in df1', file=sys.stderr)
        print('Aborting program')
        sys.exit(-1)

    return predictions


def nn_imputer(df_train, col):
    # NOT DONE ---------------------------------------------------------
    df = df_train.copy()
    
    vals = df[df[col].notnull()]
    missing = df[df[col].isnull()]
    
    X = vals.drop(columns=[col])
    y = vals[col]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    model = Sequential([
        Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=8)

    if not missing.empty:
        X_missing = missing.drop(columns=[col])
        predicted_values = model.predict(X_missing).flatten()
        df_train.loc[df_train[col].isnull(), col] = predicted_values

    # NOT DONE ---------------------------------------------------------


def get_predictionsTiered(df_train, df_test, predictors, target, epochs=100, learning_rate=0.001, l2_lambda=0.001):
    # if a column is missing more than 30% of data, drop it
    missing_percentages = df_train.isnull().mean()
    df_train = df_train.loc[:, missing_percentages < 0.3]

    # if less than 1% of data is missing, use median imputation
    for col in df_train.columns:
        if col not in predictors or missing_percentages[col] == 0:
            continue
        if missing_percentages[col] < 0.01 and missing_percentages[col] > 0:
            df_train[col].fillna(df_train[col].median(), inplace=True)
        else:  # use neural net to impute values
            nn_imputer(df_train, col)


    return get_predictions(df_train, df_test, predictors, target, epochs=100, learning_rate=0.001, l2_lambda=0.001)

        

In [16]:
# orthogonalization method
# all data is preprocessed and df test has been appended target preds
def orthogonalize(candidates, df_test, orthogonal_vars):
        orth_scores = {}
        for c in candidates:
            candset = df_test[[c, 'predicted_target']].copy().dropna() # assumes candidate has mostly non-NaN entries
            candcol = candset[c]

            X = sm.add_constant(candcol)
            temp_orth_scores = []
            for orth_var in orthogonal_vars:
                orthset = df_test[[orth_var]].copy().dropna()
                common_indices = candset.index.intersection(orthset.index)
                if common_indices.empty:
                    continue
                orth_col = orthset.loc[common_indices, orth_var]
                if np.var(orth_col) == 0:
                    print("ortho:", orth_var, "candidate", c)
                    continue # zero variance leads to divide by zero error
                candcol_common = candset.loc[common_indices, c]

                X_common = sm.add_constant(candcol_common)
                model = sm.OLS(orth_col, X_common).fit()
                temp_orth_scores.append(model.rsquared)

            if temp_orth_scores:
                orth_scores[c] = sum(temp_orth_scores) / len(temp_orth_scores)
            else:
                orth_scores[c] = 0
        return orth_scores

In [17]:
def proxy_finder(df_train, df_test, target, predictors, num_proxies=1, orth_weight=0.65, candidates=None, orthogonal_vars=None, neural_net="original", drop=True):
    if candidates is None:
        candidates = list(df_test.select_dtypes(include='number').columns) #only numerical data (don't encode categories, make user do that)


    proxy_finder_validate(target, candidates, df_train, df_test, predictors, orthogonal_vars)

    #print(f"Predictors: {predictors}") #DEBUGDEBUGDEBUG------------------------------------------------------------
    #print(f"Candidates: {candidates}")

    df_train = data_rescale(df_train, predictors, target, drop)
    df_test = data_rescale(df_test, predictors, target, drop)
    # drop any rows that are missing data from target
    df_train = df_train.dropna(subset=target)

    if neural_net == "torch":
      predicted_scores = get_predictionsTorch(df_train, df_test, predictors, target)
    elif neural_net == "tiered":
      predicted_scores = get_predictionsTiered(df_train, df_test, predictors, target)
    else:
      predicted_scores = get_predictions(df_train, df_test, predictors, target)

    df_test['predicted_target'] = predicted_scores
    #print(f"Predicted scores: {predicted_scores[:10]}")  #DEBUG DEBUG------------------------------------------------------------

    results = {}

    for c in candidates:
        candset = df_test[[c, 'predicted_target']].copy().dropna()
        if candset.empty:
            continue

        pred_scores = candset['predicted_target']
        candcol = candset[c]

        X_pred = sm.add_constant(candcol)
        model_pred = sm.OLS(pred_scores, X_pred).fit()
        results[c] = {
            'R_squared': model_pred.rsquared,
            'p_value': model_pred.pvalues[1],
            'coef': model_pred.params[1]
        }
        #print(f"candidate {c}: Results: {results}")  # Debug statement------------------------------------------------------------

    best_proxies = []

    if orthogonal_vars:
        orth_scores = orthogonalize(candidates, df_test, orthogonal_vars)
        proxy_scores = {}
        for c in candidates:
            try:
                proxy_scores[c] = (c, (1 - orth_weight) * results[c]['R_squared'] - orth_weight * orth_scores[c])
            except KeyError as e:
                continue

        sorted_results = sorted(proxy_scores.values(), key=lambda x: x[1], reverse=True)

        for i in range(min(num_proxies, len(sorted_results))):
            proxy, score = sorted_results[i]
            best_proxies.append(proxy)
            print(f"Proxy {i+1} for {target}: {proxy} with score: {score}")
    else:
        sorted_results = sorted(results.items(), key=lambda x: (-x[1]['R_squared'], x[1]['p_value']))

        for i in range(min(num_proxies, len(sorted_results))):
            proxy, metrics = sorted_results[i]
            best_proxies.append(proxy)
            print(f"Proxy {i+1} for {target}: {proxy} with R_squared: {metrics['R_squared']} and p_value: {metrics['p_value']}")

    return best_proxies

In [7]:
import warnings
warnings.filterwarnings("ignore", message="All-NaN slice encountered")
warnings.filterwarnings("ignore", category=FutureWarning, message="Series.__getitem__ treating keys as positions is deprecated")


# Suppress numpy invalid operation warnings
np.seterr(invalid='ignore')

#datafile_train =  /path/to/dftrain
#datafile_test =  /path/to/dftest
#df_train = pd.read_stata(datafile_train)
#df_test = pd.read_stata(datafile_test)



{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [8]:

target = 'christian_nationalism'  # The target variable in the training set
predictors = [ # predictors in both training and test set
                   'presvote20post',
                   'housevote20post',
                   'senvote20post',
                   'pff_jb',
                   'pff_dt',
                   'pid7',
                   'election_fairnness',
                   'educ',
                   'white',
                   'hispanic',
                   'partisan_violence',
                   'immigrant_citizenship',
                   'immigrant_deport',
                   'auth_grid_1',
                   'auth_grid_3',
                   'auth_grid_2',
                   'faminc_new'
                   ]

orthogonal_vars = [
                   'presvote20post',
                   'housevote20post',
                   'senvote20post',
                   'pff_jb',
                   'pff_dt',
                   'pid7',
                   'election_fairnness',
                   'educ',
                   'hispanic',
                   'partisan_violence',
                   'immigrant_citizenship',
                   'immigrant_deport',
                   'auth_grid_1',
                   'auth_grid_3',
                   'auth_grid_2',
                   'faminc_new']




best_proxies = proxy_finder(df_train, df_test, target, predictors, orth_weight=0.65, orthogonal_vars=orthogonal_vars, num_proxies=5, neural_net="torch", drop=False)
#print(best_proxies)
### orth weight 0.9 --> version, how many interviews, etx
### 0.85 same thing
### 0.8
### 0.75 bad
### 0.7 bad